In [ ]:
import os
print(os.getcwd())

In [ ]:
os.chdir('Dataset')

In [ ]:
print(os.getcwd())

In [ ]:
!ls

# Importing Libraries

In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from keras import backend as K
import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

# Defining Constants

In [ ]:
batch_size = 32
num_classes = 13
epochs = 225

# Loadin MATLAB File

In [ ]:
trainset = {}
trainset = sio.loadmat('chess_train_final.mat')
print("trainset size X:", trainset['X'].shape)
print("trainset size Y:", trainset['Y'].shape)

In [ ]:
trainset['X'] = np.rollaxis(trainset['X'], axis = 3)

In [ ]:
print('Trainset size X:',trainset['X'].shape)
print('Trainset size Y:',trainset['Y'].shape)

In [ ]:
label = trainset['Y']

In [ ]:
# convert class vectors to binary class matrices
trainset['Y'] = keras.utils.to_categorical(trainset['Y'], num_classes)

In [ ]:
print(trainset['Y'].shape)

In [ ]:
print(trainset['Y'].shape)
print(trainset['Y'][2606,:])

#2606 are the number of images starting from 0 to 2606 (A total of 2607 Number of Images if counted from 1)

In [ ]:
print(trainset['X'].shape[0], 'train samples')

# Setting Model Input Image Dimensions

In [ ]:
# input image dimensions
img_rows, img_cols = 150, 150
input_shape = (img_rows, img_cols, 3)

# Testing Random Image

In [ ]:
# Trainset image
#index=2606
index = np.random.randint(2606)
M1 = trainset['X'][index,:,:,:]
plt.imshow(M1)
plt.show()
y = label[index,:]

if y==0:
  classes='E'
elif y==1:
  classes='B'
elif y==2:
  classes='K'
elif y==3:
  classes='N'
elif y==4:
  classes='P'
elif y==5:
  classes='Q'
elif y==6:
  classes='R'
elif y==7:
  classes='b'
elif y==8:
  classes='k'
elif y==9:
  classes='n'
elif y==10:
  classes='p'
elif y==11:
  classes='q'
else:
  classes='r'

print('Label:',classes)

# Model Layers

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape = input_shape))
model.add(Conv2D(32, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = None, padding = 'valid'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = None, padding = 'valid'))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides = None, padding = 'valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

# Start Training the Model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(trainset['X'])

model.fit_generator(datagen.flow(trainset['X'], trainset['Y'], batch_size=32),
                    steps_per_epoch=len(trainset['X']) / 32, epochs=epochs)

# Saving Trained Model to Directory

In [ ]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'trained_model_final_bs32_225_dg_6.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)